### Pasul 2 – Selectarea celui mai bun clasificator global

In acest pas evaluam mai multi clasificatori standard pe intregul set de date, folosind doar feature-urile de intrare (release_year, limba si genul) si etichetele `movie_class` definite anterior pe baza clusterelor financiare. Scopul acestui pas este sa identificam modelul de clasificare care functioneaza cel mai bine atunci cand este antrenat global, fara impartirea setului in clustere. Acest rezultat va servi ca punct de referinta pentru comparatia cu metoda CIEL.

Feature-urile folosite sunt:
- `release_year`
- coloanele one-hot pentru `original_language`
- coloanele one-hot pentru `main_genre`

Evaluarea este realizata prin validare incrucisata stratificata cu 10 fold-uri (StratifiedKFold), astfel incat distributia claselor sa fie pastrata in fiecare fold. Metricile utilizate sunt:  
accuracy, precision macro, recall macro, F1 macro si Cohen’s kappa.

Clasificatorii testati sunt cei mentionati si in articolul CIEL:
- Support Vector Machine (kernel RBF)
- Random Forest
- Gradient Boosting

Pentru fiecare model sunt calculate scorurile medii pe fold-urile de testare. Dupa evaluarea tuturor clasificatorilor, rezultatele sunt sortate dupa F1.

La final, este selectat modelul cu cea mai buna performanta generala. Acesta reprezinta clasificatorul global de baza, care va fi comparat ulterior cu arhitectura CIEL pentru a observa daca antrenarea locala pe clustere poate aduce o imbunatatire semnificativa a performantelor.


In [1]:
# Asta citeste variabilele din memoria Jupyter
%store -r df_financial
%store -r best_params
%store -r genre_cols
%store -r lang_cols
%store -r best_row
%store -r X
%store -r y
import pandas as pd
import numpy as np
best_row = pd.read_pickle("best_row.pkl")





c:\Users\pirjo\OneDrive\Desktop\MADDM-test\MADDM-Movie-Analysis\venv\lib\site-packages\IPython\extensions\storemagic.py:148: UserWarning: This is now an optional IPython functionality, using autorestore/df_financial requires you to install the `pickleshare` library.
  obj = db["autorestore/" + arg]
c:\Users\pirjo\OneDrive\Desktop\MADDM-test\MADDM-Movie-Analysis\venv\lib\site-packages\IPython\extensions\storemagic.py:148: UserWarning: This is now an optional IPython functionality, using autorestore/best_params requires you to install the `pickleshare` library.
  obj = db["autorestore/" + arg]
c:\Users\pirjo\OneDrive\Desktop\MADDM-test\MADDM-Movie-Analysis\venv\lib\site-packages\IPython\extensions\storemagic.py:148: UserWarning: This is now an optional IPython functionality, using autorestore/genre_cols requires you to install the `pickleshare` library.
  obj = db["autorestore/" + arg]
c:\Users\pirjo\OneDrive\Desktop\MADDM-test\MADDM-Movie-Analysis\venv\lib\site-packages\IPython\extensio



### PASUL 3 – Implementarea componentei locale CIEL

In aceasta etapa este implementata metoda CIEL prin antrenarea unor clasificatori locali, cate unul pentru fiecare cluster obtinut in urma procesului de clustering pe feature-uri non-financiare. Fiecare cluster defineste o zona de expertiza, iar pentru fiecare dintre acestea este antrenat un model Random Forest folosind exclusiv datele din clusterul respectiv.

In faza de predictie, fiecare instanta este asociata clusterului din care face parte, iar predictia finala este furnizata de expertul local corespunzator acelui cluster. Astfel, decizia nu mai este luata de un singur clasificator global, ci de un ansamblu de modele specializate, fiecare adaptat caracteristicilor locale ale datelor.

Aceasta abordare permite captarea unor tipare diferite in functie de structura datelor si ofera o alternativa la modelele globale clasice. Performanta metodei CIEL este evaluata prin comparatie directa cu modelul global, folosind metrici standard precum accuracy, F1-score si Cohen’s Kappa.



In [2]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score

# ======================================================================
# PASUL 1: CREAREA "ZONELOR DE EXPERTIZA" (Input Clustering)
# ======================================================================
import pandas as pd

df_financial = pd.read_excel("cleaned_movies_metadata.xlsx")


lang_cols  = [c for c in df_financial.columns if c.startswith("original_language_")]
genre_cols = [c for c in df_financial.columns if c.startswith("main_genre_")]

# 3) Extragem anul din release_date
df_financial["release_year"] = pd.to_datetime(
    df_financial["release_date"], errors="coerce"
).dt.year

# 4) Definim feature-urile pentru CIEL (non-financiare)
features_clf = ["release_year"] + lang_cols + genre_cols

# 5) Construim X si y
X = df_financial[features_clf].copy()
y = df_financial["movie_class"].copy()

X_values = X.to_numpy()
y_values = y.to_numpy()


print("X_values shape:", X_values.shape)
print("NaN in X_values:", np.isnan(X_values).sum())

# 8) KMeans (Input Clustering)
n_ciel_clusters = 3
ciel_input_kmeans = KMeans(n_clusters=n_ciel_clusters, random_state=42, n_init=10)
input_labels = ciel_input_kmeans.fit_predict(X_values)

print(f"=== METODA CIEL: Antrenare pe {n_ciel_clusters} clustere de intrare ===")

# ======================================================================
# PASUL 2: ANTRENAREA EXPERTILOR LOCALI
# ======================================================================
### PASUL 3 – Implementarea componentei locale CIEL

# In aceasta etapa, implementez partea centrala a metodei CIEL: antrenarea 
# clasificatorilor locali pentru fiecare cluster obtinut din feature-urile 
# non-financiare. Fiecare cluster are propriul sau model specializat, 
# antrenat doar pe datele din acel grup.

experts = {}
rf_params = {
    "n_estimators": 300,           
    "random_state": 42,            
    "n_jobs": -1,                  
    "criterion": best_row['criterion'],              # Ex: 'entropy'
    "min_samples_leaf": best_row['min_samples_leaf'], # Ex: 10 
    "max_depth": None,                          # Ex: None
    "class_weight": "balanced"
}

for cluster_id in range(n_ciel_clusters):
    indices = (input_labels == cluster_id)
    
    # Folosim X_values
    X_local = X_values[indices]
    y_local = y_values[indices]


    expert = RandomForestClassifier(**rf_params)
    expert.fit(X_local, y_local)
    
    experts[cluster_id] = expert
    print(f" -> Expertul {cluster_id}: antrenat pe {len(X_local)} filme.")

# ======================================================================
# PASUL 3: PREDICTIA CIEL (Fiecare expert voteaza pe felia lui)
# ======================================================================
y_pred_ciel = np.zeros_like(y_values)

for i in range(len(X_values)):
    cluster_id = input_labels[i]
    y_pred_ciel[i] = experts[cluster_id].predict(X_values[i].reshape(1, -1))[0]

# ======================================================================
# PASUL 4: TABEL COMPARATIV DINAMIC (GLOBAL vs CIEL)
# ======================================================================

# Calculam metricile CIEL
acc_ciel = accuracy_score(y_values, y_pred_ciel)
f1_ciel = f1_score(y_values, y_pred_ciel, average='macro')
kappa_ciel = cohen_kappa_score(y_values, y_pred_ciel)

# Preluam DINAMIC metricile modelului Global
if 'best_row' in globals() and (best_row is not None):
    model_name = "Global DT (Optimized)"
    acc_global = best_row['test_accuracy']
    f1_global = best_row['test_f1']
    kappa_global = best_row['test_kappa']
else:
    print("\n ATENTIE: Variabila 'best_row' nu a fost gasita. Ruleaza celulele anterioare!")
    model_name = "Unknown Global"
    acc_global, f1_global, kappa_global = 0, 0, 0

# Afisarea tabelului
print("\n" + "="*80)
print(f"   REZULTAT FINAL: {model_name} (Global) vs. CIEL (Ansamblu)")
print("="*80)
print(f"{'METRICA':<15} | {'GLOBAL':<15} | {'CIEL':<15} | {'DIFERENTA':<15}")
print("-" * 80)

metrics_data = [
    ("Accuracy", acc_global, acc_ciel),
    ("F1-Score", f1_global, f1_ciel),
    ("Cohen's Kappa", kappa_global, kappa_ciel)
]

for metric, val_global, val_ciel in metrics_data:
    diff = val_ciel - val_global
    semn = "+" if diff > 0 else ""
    print(f"{metric:<15} | {val_global:.4f}          | {val_ciel:.4f}          | {semn}{diff:.4f}")

X_values shape: (5381, 20)
NaN in X_values: 0
=== METODA CIEL: Antrenare pe 3 clustere de intrare ===
 -> Expertul 0: antrenat pe 1585 filme.
 -> Expertul 1: antrenat pe 3420 filme.
 -> Expertul 2: antrenat pe 376 filme.

   REZULTAT FINAL: Global DT (Optimized) (Global) vs. CIEL (Ansamblu)
METRICA         | GLOBAL          | CIEL            | DIFERENTA      
--------------------------------------------------------------------------------
Accuracy        | 0.6863          | 0.4676          | -0.2187
F1-Score        | 0.3686          | 0.4315          | +0.0629
Cohen's Kappa   | 0.1141          | 0.1891          | +0.0750


In [3]:
print("GATA! Notebook rulat cu succes. Treci acum la fisierul 4_Vizualization.ipynb")

GATA! Notebook rulat cu succes. Treci acum la fisierul 4_Vizualization.ipynb
